In [ ]:
#Create initial database from csv
import csv, sqlite3

con = sqlite3.connect("db/db_tweets.db")
cur = con.cursor()
cur.execute("CREATE TABLE tweets (party, handle, tweet);")

with open('original_data/ExtractedTweets.csv','rt',encoding='utf-8') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['Party'], i['Handle'], i['Tweet']) for i in dr]

cur.executemany("INSERT INTO tweets (party, handle, tweet) VALUES (?, ?, ?);", to_db)
con.commit()
con.close()

In [ ]:
# Print tweets from db
import sqlite3

conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

def get_posts(x):
    cur.execute("SELECT * FROM tweets")
    return cur.fetchmany(x)
    
def print_tweet(t):
    print('\n@{} [{}]\n-----\n{}\n'.format(t[1], t[0], t[2]))

for tweet in get_posts(10):
    print_tweet(tweet)

conn.close()

In [ ]:
# Print tweets from db
import sqlite3
import re

conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

def analyze_mud_all():
    cur.execute("SELECT * FROM tweets")
    get_mud(cur.fetchall())

def analyze_mud(x):
    cur.execute("SELECT * FROM tweets")
    get_mud(cur.fetchmany(x))
    
def get_mud(tweets):
    mud_r = 0
    mud_d = 0
    for tweet in tweets:
        party = tweet[0]
        if party.upper() == 'DEMOCRAT':
            mud_d += find_mud(tweet)
        elif party.upper() == 'REPUBLICAN':
            mud_r += find_mud(tweet)
    print("\n\n---RESULTS---\n\nDEM MUD: {}\nREP MUD: {}".format(mud_d, mud_r))
    
def find_mud(t):
    party = t[0]
    username = t[1]
    tweet = t[2]
    regex = r""
    if party.upper() == 'DEMOCRAT':
        regex = r"(?!\brepublic\b)\brepub+|\bgop\b|\bconservative+|\bright\w*wing\b"
    elif party.upper() == 'REPUBLICAN':
        regex = r"(?!\bdemocracy\b)\bdem+|\bdnc\b|\bliberal+|\blibs+|\bleft\w*wing\b"
    #return 1 if re.search(regex, tweet, re.I)
    if(re.search(regex, tweet, re.I)):
        print("\n{}\n -- {} [{}]".format(tweet, username, party))
        return 1
    else:
        return 0

analyze_mud_all()

conn.close()